## Testing

In [19]:
# import all required libraries

import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from keras.models import load_model
import pickle

In [20]:
# global variables

model_original_path = "modelReport/"
model_center_path = "modelReport_center/"
model_object_path = "modelReport_object/"
threshold = 0.5

In [21]:
# convert probabilities to 0 and 1 predictions using threshold of 0.5

def binary_prediction(pred):
    value = 0
    count=0
    for p in range(0,len(pred)):
        for i in range(0,len(pred[p])):
            if(pred[p][i]>=threshold):
                pred[p][i]=1
            else:
                pred[p][i]=0
    return pred

### Metric 3 : Percentage correct predictions

percentage of genres correctly predicted

In [1]:
def correct_predictions_top(y_test, pred):
    count=0
    sum_count = 0
    total_correct = 0
    for p in range(0,len(pred)):
        count = len( np.where(y_test[p])[0] )
        sum_count+=count
        sorted_pred = np.sort(pred[p])
        top_pred = sorted_pred[-count:]
        #print(top_pred)
        for t in top_pred:
            ind = np.where(pred[p]==t)
            if(y_test[p][ind[0][0]]==1):
                total_correct += 1
                
    return total_correct/sum_count

### Accuracy : Metric 1 and 2

In [2]:
# Metric 1 : using confusion matrix

def accuracy_score_ones_zeros(y_test, pred):
    value = 0
    count=0
    for p in range(0,len(pred)):
        for i in range(0,len(pred[p])):
            if((pred[p][i]>=threshold and y_test[p][i]==1) or (pred[p][i]<threshold and y_test[p][i]==0)):
                count+=1
    return count/(10*len(pred))

In [4]:
# Metric 2 : using partial correctness

def accuracy_score_weighted(y_test,pred_b):
    y_true = np.array(y_test)
    y_pred = np.array(pred_b)
    acc_list=[]
    for i in range(y_true.shape[0]):
        #print(i)
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float( len(set_true.union(set_pred)))
        acc_list.append(tmp_a)
    return np.mean(acc_list)


### Precision : Metric 1 and 2

In [5]:
# Metric 1 : using confusion matrix

def precision_score_ones_zeros(y_test, pred):
    value = 0
    TP=0
    FP=0
    for p in range(0,len(pred)):
        for i in range(0,len(pred[p])):
            if(pred[p][i]>=threshold and y_test[p][i]==1):
                TP+=1
            if(pred[p][i]>=threshold and y_test[p][i]==0):
                FP+=1
    if TP+FP==0:
        return 0
    return TP/(TP+FP)

In [6]:
# Metric 2 : using partial correctness

def precision_score_weighted(y_test,pred_b):
    y_true = np.array(y_test)
    y_pred = np.array(pred_b)
    pr_list=[]
    tmp_p=0
    for i in range(y_true.shape[0]):
        #print(i)
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_p = 1
        elif len(set_pred)==0:
            tmp_p=0
        else:
            tmp_p = len(set_true.intersection(set_pred))/float(len((set_pred)))
        pr_list.append(tmp_p)
    return np.mean(pr_list)


### Recall : Metric 1 and 2

In [7]:
# Metric 1 : using confusion matrix

def recall_score_ones_zeros(y_test, pred):
    value = 0
    TP=0
    FN=0
    for p in range(0,len(pred)):
        for i in range(0,len(pred[p])):
            if(pred[p][i]>=threshold and y_test[p][i]==1):
                TP+=1
            if(pred[p][i]<threshold and y_test[p][i]==1):
                FN+=1
        
    return TP/(TP+FN)

In [8]:
# Metric 2 : using partial correctness

def recall_score_weighted(y_test,pred_b):
    y_true = np.array(y_test)
    y_pred = np.array(pred_b)
    rc_list=[]
    for i in range(y_true.shape[0]):
        #print(i)
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_r = 1
        else:
            tmp_r = len(set_true.intersection(set_pred))/float( len((set_true)))
        rc_list.append(tmp_r)
    return np.mean(rc_list)


### F1 score : Metric 1 and 2

In [9]:
# Metric 1 : using confusion matrix

def f1_score_ones_zeros(precision,recall):
    if precision+recall==0:
        return 0
    f1 = (2*precision*recall)/float(precision+recall)
    return (f1)

In [10]:
# Metric 2 : using partial correctness

def f1_score_weighted(precision,recall):
    if precision+recall==0:
        return 0
    f1 = (2*precision*recall)/float(precision+recall)
    return (f1)

## Models for Approach 1, 2 and 3

load the dataset and label pickle files for corresponding approaches

### Approach 1 : Normal Testing

In [53]:
# load the pickle files

dbfile1=open('datasetFile','rb')
dbfile2=open('yFile','rb')
db_y=pickle.load(dbfile2)
db_data=pickle.load(dbfile1)

In [11]:
# take the testing data

db_data_test = db_data[8000:]
y_test = db_y[8000:]

In [55]:
# take path for all models for approach 1

model_glob = glob.glob(model_original_path + "*.model")
model_glob

['modelReport/model_100_10.model',
 'modelReport/model_75_10.model',
 'modelReport/model_50_10.model',
 'modelReport/model_100_20.model',
 'modelReport/model_75_20.model',
 'modelReport/model_50_20.model']

In [56]:
# run predictions using all 3 metrics on Approach 1 models

pred_all = []
for model_path in model_glob:
    model = load_model(model_path)
    pred = model.predict(np.array(db_data_test))
    pred_all.append(pred)
    print()
    print('For model : ',model_path)
    print('------------NORMAL-----------')
    print('accuracy : ',accuracy_score_ones_zeros(y_test,pred) )
    precision = precision_score_ones_zeros(y_test,pred) 
    print('precision : ',precision)
    recall = recall_score_ones_zeros(y_test,pred)
    print('recall : ', recall )
    print('f1 score : ',f1_score_ones_zeros(precision,recall))
    
    print('-----------WEIGHTED-----------')
    print('accuracy : ',accuracy_score_weighted(y_test,pred) )
    precision = precision_score_weighted(y_test,pred) 
    print('precision : ',precision)
    recall = recall_score_weighted(y_test,pred)
    print('recall : ', recall )
    print('f1 score : ',f1_score_weighted(precision,recall))
    
    print('---------------Percent correct----------------')
    print('percentage correct : ',correct_predictions_top(y_test,pred))


For model :  modelReport/model_100_10.model
------------NORMAL-----------
accuracy :  0.7598671096345515
precision :  0.4032485875706215
recall :  0.17095808383233532
f1 score :  0.2401177460050463
-----------WEIGHTED-----------
accuracy :  0.18743078626799556
precision :  0.38504983388704317
recall :  0.1874861572535991
f1 score :  0.25218157399255864
---------------Percent correct----------------
percentage correct :  0.2688622754491018

For model :  modelReport/model_75_10.model
------------NORMAL-----------
accuracy :  0.7780730897009966
precision :  0
recall :  0.0
f1 score :  0
-----------WEIGHTED-----------
accuracy :  0.0
precision :  0.0
recall :  0.0
f1 score :  0
---------------Percent correct----------------
percentage correct :  0.15449101796407186

For model :  modelReport/model_50_10.model
------------NORMAL-----------
accuracy :  0.6831229235880398
precision :  0.22466281310211947
recall :  0.17455089820359282
f1 score :  0.19646166807076665
-----------WEIGHTED--------

### Approach 2 : Center Testing

In [57]:
# load the pickle files

dbfile1_center=open('datasetFile_center','rb')
dbfile2_center=open('yFile_center','rb')
db_y_center=pickle.load(dbfile2_center)
db_data_center=pickle.load(dbfile1_center)

In [58]:
# take the testing data

db_data_test = db_data_center[8000:]
y_test = db_y_center[8000:]

In [59]:
# take path for all models for approach 2

model_glob = glob.glob(model_center_path + "*.model")
model_glob

['modelReport_center/model_100_10.model',
 'modelReport_center/model_75_10.model',
 'modelReport_center/model_50_10.model',
 'modelReport_center/model_100_20.model',
 'modelReport_center/model_75_20.model',
 'modelReport_center/model_50_20.model']

In [60]:
# run predictions using all 3 metrics on Approach 2 models

pred_center_all = []
for model_path in model_glob:
    model = load_model(model_path)
    pred = model.predict(np.array(db_data_test))
    pred_center_all.append(pred)
    print()
    print('For model : ',model_path)
    print('------------NORMAL-----------')
    print('accuracy : ',accuracy_score_ones_zeros(y_test,pred) )
    precision = precision_score_ones_zeros(y_test,pred) 
    print('precision : ',precision)
    recall = recall_score_ones_zeros(y_test,pred)
    print('recall : ', recall )
    print('f1 score : ',f1_score_ones_zeros(precision,recall))
    
    print('-----------WEIGHTED-----------')
    print('accuracy : ',accuracy_score_weighted(y_test,pred) )
    precision = precision_score_weighted(y_test,pred) 
    print('precision : ',precision)
    recall = recall_score_weighted(y_test,pred)
    print('recall : ', recall )
    print('f1 score : ',f1_score_weighted(precision,recall))
    
    print('---------------Percent correct----------------')
    print('percentage correct : ',correct_predictions_top(y_test,pred))


For model :  modelReport_center/model_100_10.model
------------NORMAL-----------
accuracy :  0.6694352159468439
precision :  0.22831505483549352
recall :  0.205688622754491
f1 score :  0.21641203339108522
-----------WEIGHTED-----------
accuracy :  0.23016136687233035
precision :  0.23790223065970575
recall :  0.8600221483942415
f1 score :  0.3727054274831606
---------------Percent correct----------------
percentage correct :  0.25419161676646707

For model :  modelReport_center/model_75_10.model
------------NORMAL-----------
accuracy :  0.6257807308970099
precision :  0.20509521358723623
recall :  0.23862275449101797
f1 score :  0.22059230556324383
-----------WEIGHTED-----------
accuracy :  0.22690739861836207
precision :  0.22849760059062385
recall :  0.9620155038759689
f1 score :  0.36928318309461705
---------------Percent correct----------------
percentage correct :  0.22544910179640718

For model :  modelReport_center/model_50_10.model
------------NORMAL-----------
accuracy :  0.6

### Approach 3 : Object Detection Testing

In [31]:
# load the pickle files

dbfile1_obj=open('datasetFile_object','rb')
dbfile2_obj=open('yFile_object','rb')
db_y_object=pickle.load(dbfile2_obj)
db_data_object=pickle.load(dbfile1_obj)

In [32]:
# take the testing data

db_data_test = db_data_object[8000:]
y_test = db_y_object[8000:]

In [33]:
# take path for all models for approach 3

model_glob = glob.glob(model_object_path + "*.model")
model_glob

['modelReport_object/model_100_10.model',
 'modelReport_object/model_75_10.model',
 'modelReport_object/model_50_10.model',
 'modelReport_object/model_25_20.model',
 'modelReport_object/model_100_20.model',
 'modelReport_object/model_75_20.model',
 'modelReport_object/model_50_20.model',
 'modelReport_object/model_25_10.model']

In [34]:
# run predictions using all 3 metrics on Approach 3 models

pred_object_all = []
for model_path in model_glob:
    model = load_model(model_path)
    pred = model.predict(np.array(db_data_test))
    pred_object_all.append(pred)
    print()
    print('For model : ',model_path)
    print('------------NORMAL-----------')
    print('accuracy : ',accuracy_score_ones_zeros(y_test,pred) )
    precision = precision_score_ones_zeros(y_test,pred) 
    print('precision : ',precision)
    recall = recall_score_ones_zeros(y_test,pred)
    print('recall : ', recall )
    print('f1 score : ',f1_score_ones_zeros(precision,recall))
    
    print('-----------WEIGHTED-----------')
    print('accuracy : ',accuracy_score_weighted(y_test,pred) )
    precision = precision_score_weighted(y_test,pred) 
    print('precision : ',precision)
    recall = recall_score_weighted(y_test,pred)
    print('recall : ', recall )
    print('f1 score : ',f1_score_weighted(precision,recall))
    
    print('---------------Percent correct----------------')
    print('percentage correct : ',correct_predictions_top(y_test,pred))


For model :  modelReport_object/model_100_10.model
------------NORMAL-----------
accuracy :  0.7417275747508305
precision :  0.36010230179028135
recall :  0.21077844311377245
f1 score :  0.26591123701605285
-----------WEIGHTED-----------
accuracy :  0.22170331698570903
precision :  0.22171069978378946
recall :  0.9983388704318937
f1 score :  0.3628416664180024
---------------Percent correct----------------
percentage correct :  0.34221556886227544

For model :  modelReport_object/model_75_10.model
------------NORMAL-----------
accuracy :  0.5971428571428572
precision :  0.24081477251094613
recall :  0.3787425149700599
f1 score :  0.29442569533341095
-----------WEIGHTED-----------
accuracy :  0.2100174023097611
precision :  0.2507986605494911
recall :  0.5054263565891473
f1 score :  0.3352448024493977
---------------Percent correct----------------
percentage correct :  0.205688622754491

For model :  modelReport_object/model_50_10.model
------------NORMAL-----------
accuracy :  0.77807